<a href="https://colab.research.google.com/github/lavou/ResearchMate/blob/main/4_0_Node_Embeddings_757authors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import pandas as pd
import numpy as np
import ast

dir_path = '/content/drive/MyDrive/Pubmed_project/last_data'

DATA_PATH = os.path.join(dir_path) 
all_files = [f for f in os.listdir(DATA_PATH) if f.endswith('.csv')]
df = pd.concat([pd.read_csv(os.path.join(DATA_PATH, file), index_col=0) for file in all_files], ignore_index=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32672 entries, 0 to 32671
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Paper ID           32672 non-null  int64 
 1   Authors            32672 non-null  object
 2   Abstract           29655 non-null  object
 3   Author's Order     32672 non-null  int64 
 4   Number of Authors  32672 non-null  int64 
 5   Keywords           8834 non-null   object
 6   Title              32655 non-null  object
dtypes: int64(3), object(4)
memory usage: 1.7+ MB


# 1. Preprocess the data:

First, preprocess the data to extract unique authors and their collaborations.


In [ ]:
clean_author = df.loc[df["Author's Order"] != -1, :]
clean_author = clean_author.reset_index()
df_noOrder = clean_author.drop("Author's Order", axis=1)
df_noOrder['Authors'] = df_noOrder['Authors'].apply(lambda x: ast.literal_eval(x))
relationships = []
for idx in range(df_noOrder.shape[0]):
# Add a relationship to the list for each pair of unique characters within the window
    if len(df_noOrder.loc[idx, 'Authors']) > 1:
        for id, a in enumerate(df_noOrder.loc[idx, 'Authors'][:-1]):
            b = df_noOrder.loc[idx, 'Authors'][id + 1]
            relationships.append({"source": a, "target": b})

In [ ]:
relationship_df = pd.DataFrame(relationships)
# pd.set_option('display.max_rows', None)
# relationship_df.head()
# Sort the cases with a->b and b->a
relationship_df = pd.DataFrame(np.sort(relationship_df.values, axis = 1), columns = relationship_df.columns)
# relationship_df.head(20)

In [ ]:
# Add a new column with value 1 to the DataFrame
relationship_df["value"] = 1

# Group the DataFrame by source and target, and sum the values
relationship_df2 = relationship_df.groupby(["source", "target"], sort=False, as_index=False)["value"].sum()
# relationship_df2.head(20)

In [ ]:
# df_noOrder = df_noOrder.explode('Authors')
# df_noOrder = df_noOrder.reset_index()


In [ ]:
relationship_df2_1k = relationship_df2.loc[:1000, :]

# 2. Make the Graph

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
# Create a graph from a pandas dataframe
G = nx.from_pandas_edgelist(relationship_df2, 
                            source = "source", 
                            target = "target", 
                            edge_attr = "value", 
                            create_using = nx.Graph())

# plt.figure(figsize=(100,100))
# pos = nx.spring_layout(G)
# nx.draw(G, with_labels=True, node_color='skyblue', edge_cmap=plt.cm.Blues, pos = pos)
# plt.savefig('1k.pdf',dpi=300)
# plt.show()

# 3. Embeddings

In [ ]:
import node2vec
import numpy as np

# Create the node2vec object with adjusted hyperparameters
node2vec = node2vec.Node2Vec(G, dimensions=64, walk_length=30, num_walks=200, p=0.5, q=2, workers=4)

# Fit the model to the graph
model = node2vec.fit()

# Get the node embeddings
node_embeddings = {node: model.wv[node] for node in G.nodes()}



Computing transition probabilities:   0%|          | 0/756 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|██████████| 50/50 [00:02<00:00, 20.17it/s]


In [ ]:
#print(node_embeddings[0])
#print(G.nodes()[0])